Model Name: Google Universtal Sentence Encoder

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import tensorflow_hub as hub

In [14]:
data = pd.read_csv('c:/Users/anama_1lv/Desktop/Veridion/Company_Classifier/CSV/preped_data.csv')
labels = pd.read_csv('c:/Users/anama_1lv/Desktop/Veridion/Company_Classifier/CSV/labels.csv')

In [15]:
model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [ ]:
#less than a minute
data['d_embedding'] = data['description'].apply(lambda x: model([x]).numpy()[0] if isinstance(x, str) else None)
data['b_embedding'] = data['business_tags'].apply(lambda x: model([x]).numpy()[0] if isinstance(x, str) else None)

In [17]:
labels['l_embedding'] = labels['label'].apply(lambda x: model([x]).numpy()[0] if isinstance(x, str) else None)

In [18]:
description_similarity = []
business_tags_similarity = []

In [19]:
for i in range(data.shape[0]):
    if data['d_embedding'][i] is not None:
        description_similarity.append(cosine_similarity([data['d_embedding'][i]], labels['l_embedding'].tolist())[0])
    else:
        description_similarity.append([0] * len(labels))
    if data['b_embedding'][i] is not None:
        business_tags_similarity.append(cosine_similarity([data['b_embedding'][i]], labels['l_embedding'].tolist())[0])
    else:
        business_tags_similarity.append([0] * len(labels))

In [20]:
similarities = (np.array(description_similarity) + np.array(business_tags_similarity)) / 2

In [21]:
def get_top_labels(similarities, labels, top_n=3):
    top_indices = np.argsort(similarities, axis=1)[:, -top_n:][:, ::-1]
    top_labels_with_scores = [[(labels[i], similarities[row_idx, i]) for i in indices] for row_idx, indices in enumerate(top_indices)]
    return top_labels_with_scores

In [22]:
similarities_array = np.vstack(similarities[:data.shape[0]])

In [23]:
data['top_labels'] = get_top_labels(similarities_array, labels['label'].tolist(), top_n=3)

In [24]:
data[['description', 'business_tags', 'top_labels']].to_csv('../CSV/labelled_data_use.csv', index=False)